# **Classifier Thresholds for Likely Benign and Likely Pathogenic SNVs** 

### Author: Daniel Brock
### Date: 7/15/2023
### Purpose: to define the upper and lower threshold value for each model (percent at while predictor is 95% accurate)

In [4]:
# Importing required packages
import os
import pandas as pd
import numpy as np

# Setting working directory
cwd = os.getcwd()
print(cwd)

C:\Users\TooFastDan\OneDrive - Baylor College of Medicine\BCM\Projects\Autosomal Dominant Predictor of IRDs\manuscript\GitHub


## **Defining Functions used to calculate upper and lower threshold values for classifiers**

In [32]:
def likely_pathogenic(min_score, max_score, df, classifier, iterations=1001):
    """
    Function to calculate the upper threshold of a classifier model, where 95% of variants from the training ClinVar set are correctly classified as PLP.
    Inputs:
    - min_score: minimum score of a classifier
    - max_score: maximum score of a classifier
    - df: dataframe containing variants, classifier scores, and pathogenicity classifications
    - classifier: name of the classifier model
    Outputs:
    - scores dataframe: percent_PLP and percent_BLB for each scoring option
    - LP_threshold: threshold for a variant being likely pathogenic (LP)
    """
    
    # Creating an array of scores in the thousandths decimal place (used by classifiers)
    scores = np.linspace(min_score, max_score, num=iterations)
    scores = np.around(scores, 3)
    
    # Looping through the ClinVar variants data to get percent PLP and percent BLB for each possible classifier score
    df2 = df[df[classifier].notna()]
    scores_df_list = []
    for score in scores:
        s_df = df2[df2[classifier] >= score]
        plps = s_df[s_df["PATHOGENICITY"]=="PLP"].shape[0] #number of PLP variants classified by clinvar
        blbs = s_df[s_df['PATHOGENICITY']=="BLB"].shape[0] #number of BLB variants classified by clinvar
        pb = pd.DataFrame({'PLP': [plps], 'BLB': [blbs]})
        pb["percent_PLP"] = pb["PLP"] / (pb["PLP"] + pb["BLB"]) * 100
        pb["percent_BLB"] = pb["BLB"] / (pb["PLP"] + pb["BLB"]) * 100
        pb[classifier] = score
        scores_df_list.append(pb)
    scores_df = pd.concat(scores_df_list, axis=0)
    
    # Returning scores df and likely pathogenic threshold
    try:
        lp = min(scores_df[scores_df["percent_PLP"] > 95][classifier])
    except:
        lp = np.nan
        print("Had Issues")
    return scores_df, lp

In [13]:
def likely_benign(min_score, max_score, df, classifier, iterations=1001):
    """
    Function to calculate the lower threshold of a classifier model, where 95% of variants from the training ClinVar set are correctly classified as BLB.
    Inputs:
    - min_score: minimum score of a classifier
    - max_score: maximum score of a classifier
    - df: dataframe containing variants, classifier scores, and pathogenicity classifications
    - classifier: name of the classifier model
    Outputs:
    - scores dataframe: percent_PLP and percent_BLB for each scoring option
    - LP_threshold: threshold for a variant being likely benign (LB)
    """
    
    # Creating an array of scores in the thousandths decimal place (used by classifiers)
    scores = np.linspace(min_score, max_score, num=iterations)
    scores = np.around(scores, 3)
    
    # Looping through the ClinVar variants data to get percent PLP and percent BLB for each possible classifier score
    df2 = df[df[classifier].notna()]
    scores_df_list = []
    for score in scores:
        s_df = df2[df2[classifier] <= score]
        plps = s_df[s_df["PATHOGENICITY"]=="PLP"].shape[0] #number of PLP variants classified by clinvar
        blbs = s_df[s_df['PATHOGENICITY']=="BLB"].shape[0] #number of BLB variants classified by clinvar
        pb = pd.DataFrame({'PLP': [plps], 'BLB': [blbs]})
        pb["percent_PLP"] = pb["PLP"] / (pb["PLP"] + pb["BLB"]) * 100
        pb["percent_BLB"] = pb["BLB"] / (pb["PLP"] + pb["BLB"]) * 100
        pb[classifier] = score
        scores_df_list.append(pb)
    scores_df = pd.concat(scores_df_list, axis=0)
    
    # Returning scores df and likely pathogenic threshold
    lp = max(scores_df[scores_df["percent_BLB"] > 95][classifier])
    return scores_df, lp

# **Getting Threshold Values for ALL untrained ClinVar RetNet Genes**

### Source: Clinvar VCF File: https://www.ncbi.nlm.nih.gov/clinvar/

In [8]:
rc = pd.read_excel(cwd+"/annovar_files/clinvar_filt_retnet_annotated_PLP-BLB_Nonsynonomous_Variants.xlsx", na_values=".")
display(rc.shape)
display(rc.head())

(3330, 154)

,Chr,Start,End,Ref,Alt,QUAL,FILTER,INFO,ID,ALLELEID,...,OMIM Phenotypes,How Identified;_x000D_\nComments,Chromosome,pLI,exac_pLI,AR,AD,AD GOF,AD Haploinsuffiency,Complex
0,chr10,71142347,71142347,C,A,NaN,NaN,"ALLELEID=2671427;CLNDISDB=MONDO:MONDO:0032807,...",2506469,2671427,...,"Retinitis pigmentosa 79, 617460 (3), Autosomal...","linkage mapping, whole-exome sequencing; a mis...",10,9.145900e-01,0.555730,1,1,1,0,1.0
1,chr20,10654126,10654126,A,C,NaN,NaN,"ALLELEID=2671423;CLNDISDB=MONDO:MONDO:0016862,...",2506465,2671423,...,"?Deafness, congenital heart defects, and poste...","deletion mapping, candidate gene; multiple aff...",20,1.000000e+00,1.000000,0,1,0,1,NaN
2,chr12,48380959,48380959,C,T,NaN,NaN,"ALLELEID=2671226;CLNDISDB=MONDO:MONDO:0008702,...",2506270,2671226,...,?Vitreoretinopathy with phalangeal epiphyseal ...,"linkage mapping, candidate gene; mutations in ...",12,1.000000e+00,1.000000,0,1,0,1,NaN
3,chr20,3897663,3897663,T,A,NaN,NaN,"ALLELEID=2671068;CLNDISDB=MONDO:MONDO:0009319,...",2506108,2671068,...,"HARP syndrome, 607236 (3), Autosomal recessive...","homozygosity mapping, candidate gene; symptoms...",20,7.403400e-07,0.010792,1,0,0,0,NaN
4,chr12,48378867,48378867,C,T,NaN,NaN,ALLELEID=2670853;CLNDISDB=MedGen:CN517202;CLND...,2505889,2670853,...,?Vitreoretinopathy with phalangeal epiphyseal ...,"linkage mapping, candidate gene; mutations in ...",12,1.000000e+00,1.000000,0,1,0,1,NaN


In [36]:
# Skip if done with preprocessed data

# Importing annotated clinvar retnet variants
#rc = pd.read_csv(cwd+"/annotated_clinvar_filt_retnet_20230626.hg19_multianno.csv", na_values=".")

# Importing the clinvar retnet-filtered avinput file, which as the pathogenicity scores
#clin_avi = pd.read_table(cwd+"/clinvar_filt_retnet_20230626.avinput", sep="\t",
#                        names=['Chr', 'Start', 'End', 'Ref', 'Alt', 'QUAL', 'FILTER', 'INFO', 'ID', 'ALLELEID', 'CLNDISDB', 'CLNDN', 'CLNHGVS', 'CLNREVSTAT', 'CLNVC', 'CLNVCSO', 'GENEINFO', 'Gene', 'Gene_ID', 'MC', 'ORIGIN', 'CLNSIG', 'Pathogenicity', 'y_test'])
#clin_avi = clin_avi[['Chr', 'Start', 'End', 'Ref', 'Alt', 'Pathogenicity', 'y_test']]
#display(clin_avi.shape)
#display(clin_avi.head())

# Merging to get pathogenicity rankings
#rc = pd.merge(rc, clin_avi, on=['Chr', 'Start', 'End', 'Ref', 'Alt'])

# Filtering for nonsynonymous SNVs
#rc = rc[rc["ExonicFunc.refGene"]=="nonsynonymous SNV"]
#display(rc.shape)
#display(rc.head())

# Optional Export to excel
#rc.to_excel(cwd+"/annotated_clinvar_filt_retnet_processed.xlsx", index=False)

### MutScore Thresholds

In [14]:
# Likely Pathogenic thresholds
lp_scores, lp = likely_pathogenic(min_score=0, max_score=1, df=rc, classifier="MutScore")
display(lp_scores.head())
print("The upper likely-pathogenic threshold of MutScore is {}".format(lp))
#lp_scores.to_excel(cwd+"/retnet_classifier_thresholds/likely-pathogenic_MutScore.xlsx", index=False) #optional export to excel

# Likely Benign thresholds
lb_scores, lb = likely_benign(min_score=0, max_score=1, df=rc, classifier="MutScore")
display(lb_scores.head())
print("The lower likely-benign threshold of MutScore is {}".format(lb))
#lb_scores.to_excel(cwd+"/retnet_classifier_thresholds/likely-benign_MutScore.xlsx", index=False) #optional export to excel

,PLP,BLB,percent_PLP,percent_BLB,MutScore
0,1377,1857,42.578850,57.421150,0.000
0,1377,1857,42.578850,57.421150,0.001
0,1377,1857,42.578850,57.421150,0.002
0,1377,1857,42.578850,57.421150,0.003
0,1377,1854,42.618384,57.381616,0.004


The upper likely-pathogenic threshold of MutScore is 0.868


,PLP,BLB,percent_PLP,percent_BLB,MutScore
0,0,0,NaN,NaN,0.000
0,0,0,NaN,NaN,0.001
0,0,0,NaN,NaN,0.002
0,0,3,0.0,100.0,0.003
0,0,5,0.0,100.0,0.004


The lower likely-benign threshold of MutScore is 0.558


### VEST4

In [17]:
# Likely Pathogenic thresholds
lp_scores, lp = likely_pathogenic(min_score=0, max_score=1, df=rc, classifier="VEST4_score")
display(lp_scores.head())
print("The upper likely-pathogenic threshold of VEST4_score is {}".format(lp))
#lp_scores.to_excel(cwd+"/retnet_classifier_thresholds/likely-pathogenic_VEST4_score.xlsx", index=False) #optional export to excel

# Likely Benign thresholds
lb_scores, lb = likely_benign(min_score=0, max_score=1, df=rc, classifier="VEST4_score")
display(lb_scores.head())
print("The lower likely-benign threshold of VEST4_score is {}".format(lb))
#lb_scores.to_excel(cwd+"/retnet_classifier_thresholds/likely-benign_VEST4_score.xlsx", index=False) #optional export to excel

,PLP,BLB,percent_PLP,percent_BLB,VEST4_score
0,1378,1864,42.504627,57.495373,0.000
0,1378,1864,42.504627,57.495373,0.001
0,1378,1864,42.504627,57.495373,0.002
0,1378,1864,42.504627,57.495373,0.003
0,1378,1864,42.504627,57.495373,0.004


The upper likely-pathogenic threshold of VEST4_score is 0.906


,PLP,BLB,percent_PLP,percent_BLB,VEST4_score
0,0,0,NaN,NaN,0.000
0,0,0,NaN,NaN,0.001
0,0,0,NaN,NaN,0.002
0,0,0,NaN,NaN,0.003
0,0,0,NaN,NaN,0.004


The lower likely-benign threshold of VEST4_score is 0.463


### REVEL

In [18]:
# Likely Pathogenic thresholds
lp_scores, lp = likely_pathogenic(min_score=0, max_score=1, df=rc, classifier="REVEL_score")
display(lp_scores.head())
print("The upper likely-pathogenic threshold of REVEL_score is {}".format(lp))
#lp_scores.to_excel(cwd+"/retnet_classifier_thresholds/likely-pathogenic_REVEL_score.xlsx", index=False) #optional export to excel

# Likely Benign thresholds
lb_scores, lb = likely_benign(min_score=0, max_score=1, df=rc, classifier="REVEL_score")
display(lb_scores.head())
print("The lower likely-benign threshold of REVEL_score is {}".format(lb))
#lb_scores.to_excel(cwd+"/retnet_classifier_thresholds/likely-benign_REVEL_score.xlsx", index=False) #optional export to excel

,PLP,BLB,percent_PLP,percent_BLB,REVEL_score
0,1377,1857,42.578850,57.421150,0.000
0,1377,1857,42.578850,57.421150,0.001
0,1377,1856,42.592020,57.407980,0.002
0,1377,1855,42.605198,57.394802,0.003
0,1377,1854,42.618384,57.381616,0.004


The upper likely-pathogenic threshold of REVEL_score is 0.796


,PLP,BLB,percent_PLP,percent_BLB,REVEL_score
0,0,0,NaN,NaN,0.000
0,0,1,0.0,100.0,0.001
0,0,2,0.0,100.0,0.002
0,0,3,0.0,100.0,0.003
0,0,4,0.0,100.0,0.004


The lower likely-benign threshold of REVEL_score is 0.368


### ClinPred

In [19]:
# Likely Pathogenic thresholds
lp_scores, lp = likely_pathogenic(min_score=0, max_score=1, df=rc, classifier="ClinPred_score")
display(lp_scores.head())
print("The upper likely-pathogenic threshold of ClinPred_score is {}".format(lp))
#lp_scores.to_excel(cwd+"/retnet_classifier_thresholds/likely-pathogenic_ClinPred_score.xlsx", index=False) #optional export to excel

# Likely Benign thresholds
lb_scores, lb = likely_benign(min_score=0, max_score=1, df=rc, classifier="ClinPred_score")
display(lb_scores.head())
print("The lower likely-benign threshold of ClinPred_score is {}".format(lb))
#lb_scores.to_excel(cwd+"/retnet_classifier_thresholds/likely-benign_ClinPred_score.xlsx", index=False) #optional export to excel

,PLP,BLB,percent_PLP,percent_BLB,ClinPred_score
0,1378,1859,42.570281,57.429719,0.000
0,1378,1855,42.622951,57.377049,0.001
0,1378,1838,42.848259,57.151741,0.002
0,1378,1817,43.129890,56.870110,0.003
0,1378,1803,43.319711,56.680289,0.004


The upper likely-pathogenic threshold of ClinPred_score is 0.975


,PLP,BLB,percent_PLP,percent_BLB,ClinPred_score
0,0,4,0.0,100.0,0.000
0,0,21,0.0,100.0,0.001
0,0,42,0.0,100.0,0.002
0,0,56,0.0,100.0,0.003
0,0,76,0.0,100.0,0.004


The lower likely-benign threshold of ClinPred_score is 0.851


### MetaRNN

In [20]:
# Likely Pathogenic thresholds
lp_scores, lp = likely_pathogenic(min_score=0, max_score=1, df=rc, classifier="MetaRNN_score")
display(lp_scores.head())
print("The upper likely-pathogenic threshold of MetaRNN_score is {}".format(lp))
#lp_scores.to_excel(cwd+"/retnet_classifier_thresholds/likely-pathogenic_MetaRNN_score.xlsx", index=False) #optional export to excel

# Likely Benign thresholds
lb_scores, lb = likely_benign(min_score=0, max_score=1, df=rc, classifier="MetaRNN_score")
display(lb_scores.head())
print("The lower likely-benign threshold of MetaRNN_score is {}".format(lb))
#lb_scores.to_excel(cwd+"/retnet_classifier_thresholds/likely-benign_MetaRNN_score.xlsx", index=False) #optional export to excel

,PLP,BLB,percent_PLP,percent_BLB,MetaRNN_score
0,1381,1873,42.440074,57.559926,0.000
0,1381,1839,42.888199,57.111801,0.001
0,1381,1824,43.088924,56.911076,0.002
0,1381,1788,43.578416,56.421584,0.003
0,1381,1748,44.135507,55.864493,0.004


The upper likely-pathogenic threshold of MetaRNN_score is 0.814


,PLP,BLB,percent_PLP,percent_BLB,MetaRNN_score
0,0,34,0.0,100.0,0.000
0,0,49,0.0,100.0,0.001
0,0,85,0.0,100.0,0.002
0,0,125,0.0,100.0,0.003
0,0,193,0.0,100.0,0.004


The lower likely-benign threshold of MetaRNN_score is 0.609


### BayesDel_addAF_score

In [33]:
# Likely Pathogenic thresholds
lp_scores, lp = likely_pathogenic(min_score=-1.293, max_score=0.758, df=rc, classifier="BayesDel_addAF_score", iterations=2052)
display(lp_scores.head())
print("The upper likely-pathogenic threshold of BayesDel_addAF_score is {}".format(lp))
#lp_scores.to_excel(cwd+"/retnet_classifier_thresholds/likely-pathogenic_BayesDel_addAF_score.xlsx", index=False) #optional export to excel

# Likely Benign thresholds
lb_scores, lb = likely_benign(min_score=-1.293, max_score=0.758, df=rc, classifier="BayesDel_addAF_score", iterations=2052)
display(lb_scores.head())
print("The lower likely-benign threshold of BayesDel_addAF_score is {}".format(lb))
#lb_scores.to_excel(cwd+"/retnet_classifier_thresholds/likely-benign_BayesDel_addAF_score.xlsx", index=False) #optional export to excel

,PLP,BLB,percent_PLP,percent_BLB,BayesDel_addAF_score
0,1382,1864,42.575478,57.424522,-1.293
0,1382,1864,42.575478,57.424522,-1.292
0,1382,1864,42.575478,57.424522,-1.291
0,1382,1864,42.575478,57.424522,-1.290
0,1382,1864,42.575478,57.424522,-1.289


The upper likely-pathogenic threshold of BayesDel_addAF_score is 0.267


,PLP,BLB,percent_PLP,percent_BLB,BayesDel_addAF_score
0,0,0,NaN,NaN,-1.293
0,0,0,NaN,NaN,-1.292
0,0,0,NaN,NaN,-1.291
0,0,0,NaN,NaN,-1.290
0,0,0,NaN,NaN,-1.289


The lower likely-benign threshold of BayesDel_addAF_score is 0.013
